In [ ]:
import pandas as pd
import unicodedata
import re
import nltk

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import nltk

# Loading data via google drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_df = pd.read_csv(r"/content/drive/My Drive/data/sinhala-hate-speech-dataset.csv")
data_df = pd.DataFrame(data_df)
data_df

,id,comment,label
0,1,`හුදෙකලා´ එකේ `ද´ ඇහෙනවද කාටහරි,0
1,2,18ක පෙනුමක් තියෙන්නේ,0
2,3,2020 එනවා කිව්නාට මෙහෙම එයි කියලා හිතුවේ නෑ පට්ට,0
3,4,25 කැල්ල..හම්මෝ..,0
4,5,25 කෙල්ල අයියලගෙ කාලද කොහෙද ඉන්නෙ,1
...,...,...,...
6340,6341,ෆට්ට ඒත් ඉතින් ඔහේ ඔක්සිජන් ඔනී නැතී සෙට් එකක්...,0
6341,6342,ෆයිනලි,0
6342,6343,ෆුකෙන් හිනා මේවට...,1
6343,6344,"ෆෝන් එක චාර්ජ් කරගනින්,29%. පට්ට.....",0


In [ ]:
def lineBreakReplace(text):
    return text.replace("\r\n", " ")

In [ ]:
data_df['comment'] =data_df.apply(lambda x: lineBreakReplace(x.comment), axis=1)

In [ ]:
def removeEscapSequences(text):
    return "".join(ch for ch in text if unicodedata.category(ch)[0]!="C")

In [ ]:
data_df['comment'] =data_df.apply(lambda x: removeEscapSequences(x.comment), axis=1)

In [ ]:
def remove_digits(text):
    return ''.join([i for i in text if not i.isdigit()])

In [ ]:
data_df['comment'] =data_df.apply(lambda x: removeEscapSequences(x.comment), axis=1)

In [ ]:
def removeHtmlTags(text):
    p = re.compile(r'<.*?>')
    return p.sub('', text)

In [ ]:
data_df['comment'] = data_df.apply(lambda x: removeHtmlTags(x.comment), axis=1)

In [ ]:
data_df.head()

,id,comment,label
0,1,`හුදෙකලා´ එකේ `ද´ ඇහෙනවද කාටහරි,0
1,2,18ක පෙනුමක් තියෙන්නේ,0
2,3,2020 එනවා කිව්නාට මෙහෙම එයි කියලා හිතුවේ නෑ පට්ට,0
3,4,25 කැල්ල..හම්මෝ..,0
4,5,25 කෙල්ල අයියලගෙ කාලද කොහෙද ඉන්නෙ,1


In [ ]:
def removeSpecialCharacters(sentence,keep_apostrophes=True):
    sentence = sentence.strip()
    if keep_apostrophes:
        PATTERN = r'[?|$|&|*|%|@|(|)|~]'
        filtered_sentence = re.sub(PATTERN, r'', sentence)
        return filtered_sentence
    return sentence

In [ ]:
def replace_full_stops(sentence):
    return sentence.replace("."," ")

In [ ]:
data_df['comment'] = data_df.apply(lambda x: removeSpecialCharacters(x.comment), axis=1)
data_df['comment'] = data_df.apply(lambda x: replace_full_stops(x.comment), axis=1)

In [ ]:
def remove_english_letters(sentence):
    non_english_sentence = re.sub(r'[a-zA-Z]', '', sentence)
    return non_english_sentence

In [ ]:
data_df['comment']  = data_df['comment'] .apply(lambda x: remove_english_letters(x))

In [ ]:
lines = list(open('/content/drive/My Drive/data/stop-words-sinhala.txt',encoding="utf8"))
sinhala_stop_words = [re.sub("\n","",x) for x in lines]

In [ ]:
def tokenize_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    return tokens

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
data_df['token_list']  = data_df['comment'] .apply(lambda x: tokenize_text(x))

In [ ]:
data_df['token_list']

0               [`, හුදෙකලා´, එකේ, `, ද´, ඇහෙනවද, කාටහරි]
1                                [18ක, පෙනුමක්, තියෙන්නේ]
2       [2020, එනවා, කිව්නාට, මෙහෙම, එයි, කියලා, හිතුව...
3                                      [25, කැල්ල, හම්මෝ]
4                [25, කෙල්ල, අයියලගෙ, කාලද, කොහෙද, ඉන්නෙ]
                              ...                        
6340    [ෆට්ට, ඒත්, ඉතින්, ඔහේ, ඔක්සිජන්, ඔනී, නැතී, ස...
6341                                             [ෆයිනලි]
6342                                 [ෆුකෙන්, හිනා, මේවට]
6343                 [ෆෝන්, එක, චාර්ජ්, කරගනින්,29, පට්ට]
6344    [ෆ්ලෑශ්, ටෙලිඩ්රාමා, එක, ඇත්ත, තියරියක්, වෙන්න...
Name: token_list, Length: 6345, dtype: object

In [ ]:
def remove_sinhala_stop_words(tokens):
    filtered_tokens = [token for token in tokens if token not in sinhala_stop_words]
    return filtered_tokens

In [ ]:
data_df['token_list_after_sw'] = data_df.apply(lambda x: remove_sinhala_stop_words(x.token_list), axis=1)

In [ ]:
def remove_two_letter_words(tokenized_words):
    # Use list comprehension to filter out words with length 2
    filtered_words = [word for word in tokenized_words if len(word) > 3]
    return filtered_words

In [ ]:
data_df['token_list_after_sw'] = data_df.apply(lambda x: remove_two_letter_words(x.token_list_after_sw), axis=1)

In [ ]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
data_df['pre_processed_text'] = data_df.apply(lambda x: TreebankWordDetokenizer().detokenize(x.token_list_after_sw), axis=1)

In [ ]:
list_of_words = ' '.join(data_df['pre_processed_text']).split()

In [ ]:
from collections import Counter
import numpy as np
#most_common_words = [word for word, count in Counter(list_of_words).items() if count in np.arange(50,100)]
unique_words = [word for word, count in Counter(list_of_words).items() if count in [1,2,3]]
all_words_to_remove = unique_words;
def remove_most_common_unique_words(tokens):
    filtered_tokens = [token for token in tokens if token not in all_words_to_remove]
    return filtered_tokens

In [ ]:
data_df['token_list_after_common_words'] = data_df.apply(lambda x: remove_most_common_unique_words(x.token_list_after_sw), axis=1)

In [ ]:
data_df['pre_processed_text'] = data_df.apply(lambda x: TreebankWordDetokenizer().detokenize(x.token_list_after_common_words), axis=1)

In [ ]:
# list_of_words = ' '.join(data_df['pre_processed_text']).split()
# Counter(list_of_words)

In [ ]:
data_df['word_count'] = data_df['pre_processed_text'].str.split().str.len()
#data_df.head(40)

In [ ]:
def total_unique_words(words):
    return nltk.FreqDist(words)

print("Total number of unique words",total_unique_words(list_of_words))

Total number of unique words <FreqDist with 16629 samples and 54176 outcomes>


In [ ]:
def total_words(words):
    return(len(words))
print("Total number of words",total_words(list_of_words))

Total number of words 54176


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [ ]:
#X= data_df['pre_processed_text']
data_df["final_data"] = "__label__"+data_df['label'].astype(str) + " " + data_df["pre_processed_text"]
#data_df
#y=label_encoder.fit_transform(data_df['Type'])

In [ ]:
train, test = train_test_split(data_df, test_size=0.2, random_state=6)

In [ ]:
train.to_csv(r"/content/drive/My Drive/data/train.csv",columns=['final_data'],index=False,header=False)


In [ ]:
test.to_csv(r"/content/drive/My Drive/data/test.csv",columns=['final_data'],index=False,header=False)

In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199772 sha256=eddb146ea9a9a7f4f50831bf3590801e69e74b8f16d9a5c5c5fb6ad4d0d0bc62
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
import fasttext as ft

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.si.vec

--2024-02-01 15:40:04--  https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.si.vec
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.165.83.35, 18.165.83.79, 18.165.83.91, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.165.83.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 209303120 (200M) [binary/octet-stream]
Saving to: ‘wiki.si.vec’

wiki.si.vec         100%[===================>] 199.61M  45.6MB/s    in 4.6s    

2024-02-01 15:40:09 (43.3 MB/s) - ‘wiki.si.vec’ saved [209303120/209303120]



In [ ]:
model = ft.train_supervised(input="/content/drive/My Drive/data/train.csv", lr=0.2,loss='hs', epoch=75, wordNgrams=3, dim=300, thread=2, verbose=100,pretrainedVectors="wiki.si.vec")


In [ ]:
#classifier = ft.train_supervised(input="/content/drive/My Drive/data/train.csv",dim = 30, ws = 5, lr=1.0, epoch=30, wordNgrams=4,)

In [ ]:
out= model.test(r"/content/drive/My Drive/data/test.csv")

In [ ]:
#numberofsamples  #precision # recall
out

(1266, 0.8341232227488151, 0.8341232227488151)

In [ ]:
model.predict("25 කෙල්ල අයියලගෙ කාලද කොහෙද ඉන්නෙ",k=2)

(('__label__1', '__label__0'), array([0.91723984, 0.08278017]))